# IBM Applied Data Science Capstone Course by Coursera
### Week 5 Final Report
**_Opening a New Bakery in Delhi,India_**
- Build a dataframe of neighborhoods in Delhi,India by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new Bakery
***
### 1. Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

### 2. Loading Data from JSON File

In [2]:
with open('Delhi_neighbourhood.json') as json_data:
   delhi_data = json.load(json_data)

In [3]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods.dtypes

Borough         object
Neighborhood    object
Latitude        object
Longitude       object
dtype: object

In [4]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


### 3. Get the geographical coordinates

In [5]:
for data in delhi_data:
    borough = neighborhood_name = data['borough']
    neighborhood_name = data['neighbourhood']
    if neighborhood_name is None:
        continue
    x=neighborhood_name.find('/')
    if(x!=-1):
        neighborhood_name=neighborhood_name[:x]
    address=neighborhood_name+","+"Delhi"+",India"
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if location is None:
        continue
    neighborhood_lat = location.latitude
    neighborhood_lon =  location.longitude
#     if not( (neighborhood_lat >52.338699 and neighborhood_lat<52.676289)):
#         continue
#     if not( (neighborhood_lon >13.091450 and neighborhood_lon<13.756284)):
#         continue
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [6]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,North West Delhi,Adarsh Nagar,28.714401,77.167288
1,North West Delhi,Ashok Vihar,28.699453,77.184826
2,North West Delhi,Azadpur,28.707657,77.175547
3,North West Delhi,Bawana,28.799660,77.032885
4,North West Delhi,Dhaka,28.708698,77.205749


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 9 boroughs and 174 neighborhoods.


### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [8]:
latitude = 28.7041
longitude =77.1025
print('The geograpical coordinate of Delhi are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Delhi are 28.7041, 77.1025.


In [9]:
# create map of New York using latitude and longitude values
map_delhi = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_delhi)  
map_delhi

In [10]:
# save the map as HTML file
map_delhi.save('map_delhi.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [11]:
# define Foursquare Credentials and Version
CLIENT_ID = 'J3OL25OBHYAECQXJGRJGVVHKPDIPRZI3RAXQFQZLMURXLGN1' # your Foursquare ID
CLIENT_SECRET = 'ML4U2JXCXR5VT2S00VSX3RGRZYCAYPCRRWT3DEABREJWS2XL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J3OL25OBHYAECQXJGRJGVVHKPDIPRZI3RAXQFQZLMURXLGN1
CLIENT_SECRET:ML4U2JXCXR5VT2S00VSX3RGRZYCAYPCRRWT3DEABREJWS2XL


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [12]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [13]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7263, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adarsh Nagar,28.714401,77.167288,Shalimar bagh club,28.711882,77.157933,Indian Restaurant
1,Adarsh Nagar,28.714401,77.167288,Ever Bake,28.706419,77.162607,Bakery
2,Adarsh Nagar,28.714401,77.167288,"DT Cinemas, City Centre",28.703045,77.158189,Multiplex
3,Adarsh Nagar,28.714401,77.167288,Arjun Pao Bhaji,28.708468,77.184963,Indian Restaurant
4,Adarsh Nagar,28.714401,77.167288,Domino's Pizza,28.706000,77.163000,Pizza Place


**Let's check how many venues were returned for each neighorhood**

In [14]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adarsh Nagar,16,16,16,16,16,16
Alaknanda,54,54,54,54,54,54
Anand Vihar,31,31,31,31,31,31
Ashok Nagar,46,46,46,46,46,46
Ashok Vihar,30,30,30,30,30,30
Azadpur,29,29,29,29,29,29
Babarpur,6,6,6,6,6,6
Badarpur,12,12,12,12,12,12
Bali Nagar,48,48,48,48,48,48


**Let's find out how many unique categories can be curated from all the returned venues**

In [15]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 221 uniques categories.


In [16]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Bakery', 'Multiplex', 'Pizza Place',
       'Coffee Shop', 'Light Rail Station', 'Hotel', 'Convenience Store',
       'Restaurant', 'Bus Station', 'Gym / Fitness Center', 'Park',
       'Vegetarian / Vegan Restaurant', 'Shopping Mall',
       'Asian Restaurant', 'Snack Place', 'Fast Food Restaurant',
       'Sandwich Place', 'Chinese Restaurant', 'Department Store',
       'Donut Shop', 'Food Truck', 'South Indian Restaurant', 'BBQ Joint',
       'Dessert Shop', 'Market', 'Italian Restaurant', 'Clothing Store',
       'North Indian Restaurant', 'American Restaurant', 'Playground',
       'Café', "Men's Store", 'ATM', 'Athletics & Sports', 'Diner',
       'Arts & Crafts Store', 'Train Station', 'Music Venue',
       'Spiritual Center', 'Spa', 'Smoke Shop',
       'Mediterranean Restaurant', 'Sculpture Garden',
       'Monument / Landmark', 'Japanese Restaurant', 'History Museum',
       'Hotel Bar', 'Lounge', 'Bistro'], dtype=object)

In [17]:
# check if the results contain "Bakery"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [18]:
# one hot encoding
Delhi_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Delhi_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Delhi_onehot.columns[-1]] + list(Delhi_onehot.columns[:-1])
Delhi_onehot = Delhi_onehot[fixed_columns]

print(Delhi_onehot.shape)
Delhi_onehot.head()

(7263, 222)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adarsh Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [19]:
Delhi_grouped = Delhi_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(Delhi_grouped.shape)
Delhi_grouped

(164, 222)


,Neighborhoods,ATM,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burmese Restaurant,Bus Station,Business Service,Cafeteria,Café,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hindu Temple,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Karnataka Restaurant,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Luggage Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Nightclub,Nightlife Spot,North Indian Restaurant,Northeast Indian Restaurant,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Punjabi Restaurant,Racetrack,Recreation Center,Rental Car Location,Restaurant,River,Road,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Sushi Restaurant,Tapas Restaurant,Tea Room,Temple,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Tibetan Restaurant,Toll Booth,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Women's Store,Yoga Studio,Zoo
0,Adarsh Nagar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [20]:
len(Delhi_grouped[Delhi_grouped["Bakery"] > 0])

74

**Create a new DataFrame for Bakery data only**

In [21]:
Delhi_bakery = Delhi_grouped[["Neighborhoods","Bakery"]]

In [22]:
Delhi_bakery.head()

,Neighborhoods,Bakery
0,Adarsh Nagar,0.062500
1,Alaknanda,0.018519
2,Anand Vihar,0.032258
3,Ashok Nagar,0.021739
4,Ashok Vihar,0.033333


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [23]:
# set number of clusters
kclusters = 4

Delhi_clustering = Delhi_bakery.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Delhi_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 2, 2, 2, 0, 0, 2, 2], dtype=int32)

In [24]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Delhi_merged = Delhi_bakery.copy()

# add clustering labels
Delhi_merged["Cluster Labels"] = kmeans.labels_

In [25]:
Delhi_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
Delhi_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,Adarsh Nagar,0.062500,1
1,Alaknanda,0.018519,2
2,Anand Vihar,0.032258,2
3,Ashok Nagar,0.021739,2
4,Ashok Vihar,0.033333,2


In [26]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Delhi_merged = Delhi_merged.join(neighborhoods.set_index("Neighborhood"), on="Neighborhood")

print(Delhi_merged.shape)
Delhi_merged.head() # check the last columns!

(174, 6)


,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
0,Adarsh Nagar,0.062500,1,North West Delhi,28.714401,77.167288
1,Alaknanda,0.018519,2,South Delhi,28.529336,77.251632
2,Anand Vihar,0.032258,2,East Delhi,28.641115,77.312502
3,Ashok Nagar,0.021739,2,West Delhi,28.636021,77.101822
4,Ashok Vihar,0.033333,2,North West Delhi,28.699453,77.184826


In [27]:
# sort the results by Cluster Labels
print(Delhi_merged.shape)
Delhi_merged.sort_values(["Cluster Labels"], inplace=True)
Delhi_merged

(174, 6)


,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
83,Maujpur,0.000000,0,North East Delhi,28.692078,77.279697
112,Pitam Pura,0.000000,0,North West Delhi,28.703268,77.132250
111,Patparganj,0.000000,0,East Delhi,28.611592,77.290564
110,Patel Nagar,0.000000,0,West Delhi,28.659809,77.156957
109,Paschim Vihar,0.000000,0,West Delhi,28.669578,77.095956
107,Palam,0.000000,0,South West Delhi,28.591893,77.082824
105,Okhla,0.000000,0,South Delhi,28.563662,77.289055
103,New Usmanpur,0.000000,0,North East Delhi,28.683855,77.256005
100,Nehru Vihar,0.000000,0,North Delhi,28.710745,77.221174
98,Nehru Nagar,0.010000,0,West Delhi,28.568511,77.251385


**Finally, let's visualize the resulting clusters**

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Delhi_merged['Latitude'], Delhi_merged['Longitude'], Delhi_merged['Neighborhood'], Delhi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [29]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [30]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 0]

,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
83,Maujpur,0.000000,0,North East Delhi,28.692078,77.279697
112,Pitam Pura,0.000000,0,North West Delhi,28.703268,77.132250
111,Patparganj,0.000000,0,East Delhi,28.611592,77.290564
110,Patel Nagar,0.000000,0,West Delhi,28.659809,77.156957
109,Paschim Vihar,0.000000,0,West Delhi,28.669578,77.095956
107,Palam,0.000000,0,South West Delhi,28.591893,77.082824
105,Okhla,0.000000,0,South Delhi,28.563662,77.289055
103,New Usmanpur,0.000000,0,North East Delhi,28.683855,77.256005
100,Nehru Vihar,0.000000,0,North Delhi,28.710745,77.221174
98,Nehru Nagar,0.010000,0,West Delhi,28.568511,77.251385


#### Cluster 1

In [31]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 1]

,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
26,Dhaka,0.052632,1,North West Delhi,28.708698,77.205749
141,Shalimar Bagh,0.052632,1,North West Delhi,28.717453,77.150867
115,Preet Vihar,0.058824,1,East Delhi,28.641441,77.295259
159,Vikaspuri,0.041667,1,West Delhi,28.638419,77.070836
35,Geeta Colony,0.055556,1,East Delhi,28.650101,77.275921
0,Adarsh Nagar,0.062500,1,North West Delhi,28.714401,77.167288
162,Wazirabad,0.037736,1,North Delhi,28.433762,77.087757
87,Model Town,0.038462,1,North West Delhi,28.702714,77.193991
51,Jangpura,0.044944,1,South Delhi,28.582457,77.241500
50,Janakpuri,0.041667,1,West Delhi,28.621927,77.087476


#### Cluster 2

In [32]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 2]

,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
15,Chawri Bazaar,0.020000,2,North Delhi,28.649265,77.226515
80,Mahipalpur,0.022222,2,South West Delhi,28.544485,77.125691
140,Shakti Nagar,0.027778,2,North Delhi,28.679790,77.194914
140,Shakti Nagar,0.027778,2,North Delhi,28.679790,77.194914
57,Kailash Colony,0.030000,2,South Delhi,28.553052,77.242969
78,Lodi Colony,0.014085,2,South Delhi,28.590702,77.220921
58,Kalkaji,0.015152,2,South Delhi,28.537070,77.261805
59,Kamla Nagar,0.027027,2,North Delhi,28.680344,77.202129
144,Shivaji Place,0.016949,2,West Delhi,28.651657,77.121703
145,Siri Fort,0.030000,2,South Delhi,28.552146,77.224698


#### Cluster3


In [33]:
Delhi_merged.loc[Delhi_merged['Cluster Labels'] == 3]

,Neighborhood,Bakery,Cluster Labels,Borough,Latitude,Longitude
90,Mukherjee Nagar,0.1,3,North East Delhi,28.712557,77.214404


#### Observations:
Most of the Bakery are concentrated in the central area of Delhi city, with the highest number in cluster 3 and moderate number in cluster 2 and cluster 1. On the other hand, cluster 0 has very low number to totally no Bakery in the neighborhoods. This represents a great opportunity and high potential areas to open new Bakery as there is very little to no competition from existing malls. Meanwhile, Bakery in cluster 3 are likely suffering from intense competition due to oversupply and high concentration of Bakery. From another perspective, this also shows that the oversupply of Bakery mostly happened in the central area of the city, with the suburb area still have very few Bakery. Therefore, this project recommends Bakers to capitalize on these findings to open new Bakery in neighborhoods in cluster 0 with little to no competition. Baker with unique selling propositions to stand out from the competition can also open new Bakery in neighborhoods in cluster 1 with moderate competition. Lastly, Baker are advised to avoid neighborhoods in cluster 3 which already have high concentration of Bakery and suffering from intense competition.